In [10]:
%cd ~/google_drive/code/projects/sparse/python
from sparse.core.sparse_dataframe import SparseDataFrame
from sparse.utilities.utils import *

%cd ~/google_drive/code/projects/texture_classifier/python
# %cd /home/ubuntu/texture_classifier/python
import multiprocessing
import cPickle
import PIL
from sklearn.cluster import KMeans
from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist, squareform
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
import cv2
from pandas.io.pytables import HDFStore
# from plotly import plotly
# from plotly.graph_objs import *

import core.utils
reload(core.utils)
from core.utils import *

import core.image_scanner
reload(core.image_scanner)
from core.image_scanner import ImageScanner

import core.pipeline
reload(core.pipeline)
from core.pipeline import *

/Users/alexbraun/google_drive/code/projects/sparse/python
/Users/alexbraun/google_drive/code/projects/texture_classifier/python


In [ ]:
def archive(info, hdf_path, features=['r', 'g', 'b', 'h', 's', 'v', 'fft_max', 'fft_std']):
    hdf = HDFStore(hdf_path)
    data = get_data(info, features=features)
    x = data.drop('y', axis=1)
    y = data.y
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2)
    hdf['train_x'] = DataFrame(train_x)
    hdf['test_x'] = DataFrame(test_x)
    hdf['train_y'] = Series(train_y)
    hdf['test_y'] = Series(test_y)
    hdf.close()
    return train_x, test_x, train_y, test_y

def majority_vote(y_true, y_pred):
    d = DataFrame([y_true, y_pred]).T
    d.columns = ['ytrue', 'yhat']
    lut = {k:i for i, k in enumerate(wood_mask)}
    ilut = {v:k for k, v in lut.iteritems()}
    d.yhat = d.yhat.apply(lambda x: lut[x])
    d = d.groupby('ytrue').agg(lambda x: x.mode()).yhat.apply(lambda x: ilut[x])
    d = DataFrame(d)
    print(d[d.index == d.yhat].shape[0] / float(d.shape[0]))
    return d

def predict(info, estimator):
    y = []
    for i, row in info.iterrows():
        x = get_data(row.to_frame().T)
        x = x.drop('y', axis=1)
        x = Series(estimator.predict(x)).mode().values[0]
        y.append(x)
    return Series(y)

In [2]:
source = '/Users/alexbraun/Documents/data/texture_classifier/data/texture'
spec = [
    'material',
    'image_id',
    'label',
    'origin',
    'desc',
    'extension'
]

wood_mask = [
    'olive',
    'european-ash',
    'african-mahogany',
    'bamboo',
    'sapele',
    'kingwood',
    'teak'
#     'european-lime',
#     'makore'
]

In [3]:
# random scan
# min_res = 10
# max_res = 100
# PARAMS = {
#             'scan_method':      'random_scan',
#             'min_resolution':   (min_res, min_res),
#             'max_resolution':   (max_res, max_res),
#             'patches':          100,
#             'patch_resolution': (max_res, max_res),
# #             'normalize':        True,
#             'bins':             256
# #             'rotation':         'random'
# }

# grid scan
min_res = 25
max_res = 100
PARAMS = {
            'scan_method':      'grid_scan',
            'min_resolution':   (min_res, min_res),
            'max_resolution':   (max_res, max_res),
            'resolutions':      4,
            'spacing':          'even',
#             'patch_resolution': (min_res, min_res),
#             'normalize':        True,
            'bins':             256
}

info = get_info(source, spec, '\.')
info = info[info.label.apply(lambda x: x in wood_mask)]

# dataframes won't allow direct assignment of dicts
info['params'] = None
info.params = info.params.apply(lambda x: PARAMS)

info = info[info.desc.apply(lambda x: '_d_' in x)]

# info = info[info.desc.apply(lambda x: '_a_' in x)]
info.label = info.label.apply(lambda x: re.sub('--.*', '', x))

# train, test = info_split(info)

# train_info = info[info.desc.apply(lambda x: '_d_' in x)]
# test_info = info[info.origin == 'wood-database']i

# train_x, test_x, train_y, test_y = archive(info, hdf_path)

# info = info[info.origin != 'arroway-textures']
# info = info.head(20)

In [165]:
!rm -rf /var/tmp/hdf_batch/

In [5]:
# %%snakeviz

version = '13'.zfill(3)
hdf_path = '/Users/alexbraun/Documents/data/texture_classifier/data/hdf/data.'
hdf_path += version + '.aw-d_wood-mask.10x10-100x100.4-even.grid.hdf'



# train_x, valid_x, test_x, train_y, valid_y, test_y = archive_data(train_info, test_info, hdf_path)
# %time train_x, test_x, train_y, test_y = archive_data(train, test, hdf_path=hdf_path, cross_val=False)
# train_x, valid_x, test_x, train_y, valid_y, test_y = read_archive(hdf_path)
# data = _batch_get_data(info)



# data = get_data(info, hdf_path, processes=23)
data = pd.read_hdf(hdf_path)
X = data.drop('y', axis=1)
y = data.y
train_x, test_x, train_y, test_y = train_test_split(X, y)

In [177]:
clf = SVC()
params = {
    'C':            [1], #np.arange(0.1, 1, 0.1),
    'kernel':       ['linear'], #, 'rbf'],
#     'degree':       [3],
#     'gamma':        [0.0],
#     'coef0':        [0.0],
#     'shrinking':    [True],
#     'probability':  [False],
#     'tol':          [0.001],
#     'cache_size':   [200],
#     'class_weight': [None],
#     'verbose':      [False],
    'max_iter':     [-1],
#     'random_state': [None]
}

grid = GridSearchCV(clf, params, cv=5)
%time grid.fit(train_x, train_y)
print(grid.best_params_)
# pred = grid.best_estimator_.predict(valid_x)
# valid_report = get_report(valid_y, pred)
# valid_report
pred = grid.best_estimator_.predict(test_x)
print(grid.best_estimator_.score(test_x, test_y))
report = get_report(test_y, pred)
report

CPU times: user 10min 50s, sys: 4.33 s, total: 10min 55s
Wall time: 10min 55s
{'kernel': 'linear', 'C': 1, 'max_iter': -1}
0.180443164005


,precision,recall,f1-score,support
0,,,,
african-mahogany,0.16,0.09,0.12,3434
bamboo,0.00,0.00,0.00,3258
european-ash,0.18,0.25,0.21,3394
kingwood,1.00,0.04,0.08,3323
olive,0.19,0.55,0.28,3360
sapele,0.16,0.24,0.19,3377
teak,0.18,0.08,0.11,3457
total,0.27,0.18,0.14,23603


In [6]:
clf = RandomForestClassifier()
params = {
    'n_estimators':        [100], #[1000],
#     'criterion':           ['gini'],
    'max_depth':           [None],
#     'min_samples_split':   [2],
#     'min_samples_leaf':    [1],
    'max_features':        [38], 
#     'max_leaf_nodes':      [None],
#     'bootstrap':           [True],
#     'oob_score':           [False],
    'n_jobs':              [-1]
#     'random_state':        [42]
#     'verbose':             [0],
#     'min_density':         [None],
#     'compute_importances': [None]
}

# train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.5)

grid = GridSearchCV(clf, params, cv=5)
%time grid.fit(train_x, train_y)
print(grid.best_params_)
# pred = grid.best_estimator_.predict(valid_x)
# valid_report = get_report(valid_y, pred)
# valid_report
pred = grid.best_estimator_.predict(test_x)
print(grid.best_estimator_.score(test_x, test_y))
report = get_report(test_y, pred)
report

CPU times: user 4min 55s, sys: 1.26 s, total: 4min 56s
Wall time: 16.3 s
{'max_features': 38, 'n_estimators': 100, 'n_jobs': -1, 'max_depth': None}
0.999915268599


,precision,recall,f1-score,support
0,,,,
african-mahogany,1.00,1.00,1.00,1715
bamboo,1.00,1.00,1.00,1642
european-ash,1.00,1.00,1.00,1699
kingwood,1.00,1.00,1.00,1655
olive,1.00,1.00,1.00,1686
sapele,1.00,1.00,1.00,1685
teak,1.00,1.00,1.00,1720
total,1.00,1.00,1.00,11802


In [9]:
def func():
#     pkl = '/Users/alexbraun/google_drive/code/projects/texture_classifier/web/database/models/'
    pkl = '/Users/alexbraun/Desktop/'
    pkl += 'random-forest.arroway-d_7-woods.grid.10x10-100x100.4-even.100-est.pkl'
    with open(pkl, 'w') as a:
        cPickle.dump(grid.best_estimator_, a)
    #     pickle.dump(p, a)
    
%time func()

CPU times: user 620 ms, sys: 32.9 ms, total: 653 ms
Wall time: 676 ms


In [33]:
pkl = '/Users/alexbraun/google_drive/code/projects/texture_classifier/web/database/models/random-forest.arroway-d.grid.100x100.pkl'
p = None
with open(pkl, 'r') as l:
    p = cPickle.load(l)

In [19]:
import model
reload(model)
from model import *
# ------------------------------------------------------------------------------

PARAMS = {
    'scan_method':      'grid_scan',
    'min_resolution':   (100, 100),
    'max_resolution':   (100, 100),
    'resolutions':      1,
    'spacing':          'even',
    # 'patch_resolution': (100, 100),
    # 'normalize':        True,
    'bins':             256
}

IMAGE_SPEC = [
    'material',
    'image_id',
    'label',
    'origin',
    'descriptor',
    'extension'
]

SEP = '\.'
# ------------------------------------------------------------------------------

# def process_data(info, features=['r', 'g', 'b', 'h', 's', 'v', 'fft_std', 'fft_max']):
#     # create data from info
#     data = info.copy()
#     data.reset_index(drop=True, inplace=True)
#     data = data[['source', 'label', 'params']]
    
#     err = data.source.tolist()

#     data.source = data.source.apply(lambda x: PIL.Image.open(x))
#     data = data.apply(lambda x: 
#         generate_samples(x['source'], x['label'], x['params']),
#         axis=1
#     )
#     # create new expanded dataframe
#     data = list(chain(*data.tolist()))
#     data = DataFrame(data, columns=['x', 'y', 'params'])
#     data['bgr'] = data.x.apply(pil_to_opencv)
    
#     del data['x']
    
#     # create feature lists
#     rgb = filter(lambda x: x in list('rgb'), features)
#     hsv = filter(lambda x: x in list('hsv'), features)
#     fft = filter(lambda x: x in ['fft_std', 'fft_max'], features)
    
#     # rgb distributions
#     if rgb:
#         temp = data[['bgr', 'params']].apply(lambda x: (x['bgr'], x['params']), axis=1)
#         for chan in rgb:
#             chan_data = temp.apply(lambda x: get_channel_histogram(x[0], chan, **x[1]))
#             # data[chan] = chan_data.apply(lambda x: x.tolist())
#             create_histogram_stats(data, chan_data, chan)
            
#     # hsv distributions
#     if hsv:
#         try:
#             data['hsv'] = data.bgr.apply(lambda x: cv2.cvtColor(x, cv2.COLOR_BGR2HSV))
#         except:
#             print(err)
#             raise SyntaxError
#         temp = data[['hsv', 'params']].apply(lambda x: (x['hsv'], x['params']), axis=1)
#         for chan in hsv:
#             chan_data = temp.apply(lambda x: get_channel_histogram(x[0], chan, **x[1]))
#             # data[chan] = chan_data.apply(lambda x: x.tolist())
#             create_histogram_stats(data, chan_data, chan)
    
#         del data['hsv']
    
#     # grain frequency
#     if fft:
#         data['gray'] = data.bgr.apply(lambda x: cv2.cvtColor(x, cv2.COLOR_BGR2GRAY))
#         data.gray = data.gray.apply(lambda x: np.fft.hfft(x).astype(float))
#         data.gray = data.gray.apply(lambda x: np.histogram(x.ravel(), bins=256)[0])
#         if 'fft_std' in fft:
#             data['fft_std'] = data.gray.apply(lambda x: x.std())
#         if 'fft_max' in fft:
#             data['fft_max'] = data.gray.apply(lambda x: x.max())

#         del data['gray']
    
#     del data['bgr']
#     del data['params']
    
#     # expand columns that contain lists
#     # if rgb or hsv:
#     #     data = _flatten(data)

#     # shuffle data to destroy serial correlations
#     index = data.index.tolist()
#     np.random.shuffle(index)
#     data = data.ix[index]
#     data.reset_index(drop=True, inplace=True)

#     # Normalize features
#     cols = data.drop('y', axis=1).columns.tolist()
#     ss = StandardScaler()
#     data[cols] = ss.fit_transform(data[cols])
    
#     return data

# def create_histogram_stats(data, chan_data, channel):
#     data[channel + '_' + 'mean']   = chan_data.apply(lambda x: x.mean() )
#     data[channel + '_' + 'max']    = chan_data.apply(lambda x: x.max() )
#     data[channel + '_' + 'argmax'] = chan_data.apply(lambda x: np.argmax(x) )
#     data[channel + '_' + 'std']    = chan_data.apply(lambda x: x.std() )
#     data[channel + '_' + 'skew']   = chan_data.apply(lambda x: scipy.stats.skew(x) )
#     data[channel + '_' + 'kurt']   = chan_data.apply(lambda x: scipy.stats.kurtosis(x) )

# class TextureClassifier(object):
#     def __init__(self, db_path, model_name):
#         self._db_path = db_path
#         self._model_path = os.path.join(db_path, 'models')
#         self._image_path = os.path.join(db_path, 'images')
#         self._desc_path = os.path.join(db_path, 'descriptions.json')
#         self._temp_path = os.path.join(db_path, 'temp')
#         self.set_model(model_name)      
    
#     def set_model(self, model):
#         self._model = model
        
#     @property
#     def info(self):
#         desc = None
#         with open(self._desc_path, 'r') as d:
#             desc = json.load(d)

#         info = get_info(self._image_path, IMAGE_SPEC, sep=SEP)
#         info['description'] = info.label.apply(
#             lambda x: desc[x] if desc.has_key(x.lower()) else None) 
#         return info

#     def get_data(self, fullpath):
#         info = get_info(fullpath)
#         info['label'] = 'unknown'
#         info['params'] = None
#         info.params = info.params.apply(lambda x: PARAMS)
#         data = process_data(info).drop('y', axis=1)
#         return data

#     def get_results(self, pred):
#         data = pred.merge(self.info, how='inner', on='label')
#         data.drop_duplicates('label', inplace=True)
#         data = data.apply(lambda x: x.to_dict(), axis=1).tolist()
#         return data

#     def predict(self, filepath):
#         pred = self.get_data(filepath)
#         pred = self._model.predict(pred)
#         pred = compile_predictions(pred)
#         pred = self.get_results(pred)
#         return pred
    
db_path = '/Users/alexbraun/google_drive/code/projects/texture_classifier/web/database'
model_name = 'random-forest.arroway-d_7-woods.grid.10x10-100x100.4-even.100-est.pkl'
self = TextureClassifier(db_path, model_name)

filepath = info.loc[1445, 'source']

pred = self.get_data(filepath)
pred = self._model.predict(pred)
pred = compile_predictions(pred)
pred = self.get_results(pred)
pred

[]

In [241]:
def create_histogram_stats(data, chan_data, channel):
    data[channel + '_' + 'mean']   = chan_data.apply(lambda x: x.mean() )
#     data[channel + '_' + 'max']    = chan_data.apply(lambda x: x.max() )
#     data[channel + '_' + 'argmax'] = chan_data.apply(lambda x: np.argmax(x) )
#     data[channel + '_' + 'std']    = chan_data.apply(lambda x: x.std() )
#     data[channel + '_' + 'skew']   = chan_data.apply(lambda x: scipy.stats.skew(x) )
#     data[channel + '_' + 'kurt']   = chan_data.apply(lambda x: scipy.stats.kurtosis(x) )


def process_data(info, features=['r', 'g', 'b', 'h', 's', 'v', 'fft_std', 'fft_max']):
    # create data from info
    data = info.copy()
    data.reset_index(drop=True, inplace=True)
    data = data[['source', 'label', 'params']]
    
    err = data.source.tolist()

    data.source = data.source.apply(lambda x: PIL.Image.open(x))
    data = data.apply(lambda x: 
        generate_samples(x['source'], x['label'], x['params']),
        axis=1
    )
    # create new expanded dataframe
    data = list(chain(*data.tolist()))
    data = DataFrame(data, columns=['x', 'y', 'params'])
    data['bgr'] = data.x.apply(pil_to_opencv)
    
    del data['x']
    
    # create feature lists
    rgb = filter(lambda x: x in list('rgb'), features)
    hsv = filter(lambda x: x in list('hsv'), features)
    fft = filter(lambda x: x in ['fft_std', 'fft_max'], features)
    
    # rgb distributions
    if rgb:
        temp = data[['bgr', 'params']].apply(lambda x: (x['bgr'], x['params']), axis=1)
        for chan in rgb:
            chan_data = temp.apply(lambda x: get_channel_histogram(x[0], chan, **x[1]))
            # data[chan] = chan_data.apply(lambda x: x.tolist())
            create_histogram_stats(data, chan_data, chan)
            
    return data

i = get_info(y)
i['label'] = 'unknown'
i['params'] = None
i.params = i.params.apply(lambda x: PARAMS)
d = process_data(i).drop('y', axis=1)
d.r_mean.unique()

array([ 2.44140625])

In [240]:
min_res = 25
max_res = 100
PARAMS = {
            'scan_method':      'grid_scan',
            'min_resolution':   (min_res, min_res),
            'max_resolution':   (max_res, max_res),
            'resolutions':      4,
            'spacing':          'even',
#             'patch_resolution': (min_res, min_res),
#             'normalize':        True,
            'bins':             256
}